<a href="https://colab.research.google.com/github/TERAKOSHITAISUKE/TERAKOSHITAISUKE/blob/main/deep_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable-Baseline 3 を使った深層強化学習

In [1]:
# 必要なライブラリ類のインストール
!apt -qq update && apt -qq install xvfb python-opengl 
!pip install -q gym stable-baselines3[extra]
!pip install Box2D box2d-py pybullet gym-notebook-wrapper -q

39 packages can be upgraded. Run 'apt list --upgradable' to see them.
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl xvfb
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 1,281 kB of archives.
After this operation, 7,686 kB of additional disk space will be used.
Selecting previously unselected package python-opengl.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Selecting previously unselected package xvfb.
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.9_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.9) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Setting up xvfb (2:1.19.6-1ubuntu4.9) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 174 kB 5.4 MB/s 
     |████████████████████████████████| 1.3 MB 5.4 MB/s 
     |███████████████████

In [2]:
from datetime import datetime, timedelta, timezone

# 環境に関するライブラリ
import gym # OpenAI Gym
import gnwrapper # 結果を動画にするためのライブラリ
import pybullet_envs # 物理演算などを行うライブラリ（gym内部で使用）

# 深層強化学習に関するライブラリ
from stable_baselines3 import A2C, DQN, PPO # 深層強化学習手法
from stable_baselines3.common.env_util import make_vec_env # 学習の並列化

## OpenAI Gym の利用方法

In [3]:
# 固有のタスク名
task_name = "LunarLander-v2"

# OpenAI Gym 環境を構築
env = gym.make(task_name)
env.seed(1234) # 結果を固定化するためのシード値指定

# 動画保存のために、JSTで日付・時間を取得
jst = timezone(timedelta(hours=9), 'JST') # タイムゾーンの設定
now_datetime = datetime.now(jst).strftime("%Y%m%d-%H%M%S") # 現時刻を 年月日-時分秒 の形式で出力

# 結果を動画にするためのラッパーを設定
env = gnwrapper.Monitor(env, # もとの環境
                        directory=f"./animation/{now_datetime}") # 動画の保存先 

In [4]:
# 1エピソードをお試しで実行
state = env.reset() # 環境を初期化し、初期状態を取得
while True:
    # 行動の取得（今回はランダム）
    action = env.action_space.sample() 
    # 行動を渡して次状態, 報酬, エピソードの終了フラッグを取得
    state, reward, is_done, _ = env.step(action) 
    # 動画のフレームを追加
    env.render()
    # 終了フラッグ id_done が True なら終了
    if is_done:
        break

# 保存した描画データをアニメーションとして表示する
env.display() 

'openaigym.video.0.72.video000000.mp4'

## stable-baselines3 で対応している手法
| **手法の名称**       | `Box`          | `Discrete`     | `MultiDiscrete` | `MultiBinary`  | **並列化対応**              |
| ------------------- | ------------------ | ------------------ | ------------------- | ------------------ | --------------------------------- |
| A2C    | ✅ | ✅ | ✅ | :✅ | ✅                |
| DDPG  | ✅ | ❌               | ❌                | ❌                | ❌                               |
| DQN   | ❌ | ✅ | ❌                 | ❌                | ❌                               |
| HER   | ✅ | ✅ | ❌                 | ❌                | ❌                               |
| PPO   | ✅ | ✅ | ✅ | ✅ | ✅                |
| SAC    | ✅ | ❌                | ❌                 | ❌                | ❌                               |
| TD3   | ✅ | ❌                | ❌                 | ❌                | ❌                               |

In [5]:
# 使用できる手法は，OpenAI Gym環境のaction_spaceの型に依存
## action_spaceの型が，Boxなら行動は連続値，Discreteなら行動は離散値，など
type(env.action_space)

gym.spaces.discrete.Discrete

## 学習したポリシーを評価する関数

In [6]:
def evaluate_trained_policy(model, # 学習したモデル（方策）
                            task_name, # タスク名
                            seed=1234): # シード値
    # 環境の設定（先程と同じ）
    jst = timezone(timedelta(hours=9), 'JST')
    now_datetime = datetime.now(jst).strftime("%Y%m%d-%H%M%S")
    env = gym.make(task_name)
    env.seed(seed)
    env = gnwrapper.Monitor(env,
                             directory=f"./animation/{now_datetime}_eval")

    # エピソード実行
    state = env.reset()
    while True:
        action, _ = model.predict(state, deterministic=True) # モデルに状態を渡して行動を取得
        state, reward, is_done, _ = env.step(action)
        env.render() 
        if is_done:
            break

    env.display()

## DQN｜Deep Q-Network

In [7]:
# DQN モデルの作成
dqn = DQN("MlpPolicy", # 方策の種類（入力が普通の値なら MlpPolicy, 画像ならCnnPolicy）
                env, # 学習に用いる環境
                learning_rate=0.0001, # 学習率（デフォルト値）
                seed=1234) # 再現性のための seed 値

# 学習（約3分）
dqn.learn(total_timesteps=100000)

In [8]:
# 学習した方策で1エピソード実行
evaluate_trained_policy(dqn, task_name)

'openaigym.video.1.72.video000000.mp4'

## PPO｜Proximal Policy Optimization

In [9]:
# CPU コア単位で並列化
pararell_env = make_vec_env(task_name, n_envs=4)

# PPO モデルを作成
ppo = PPO("MlpPolicy", # 方策の種類
                pararell_env, # 並列化した環境
                learning_rate=0.0003, # 学習率（デフォルト値）
                seed=1234) # シード値

# 学習（約2分）
ppo.learn(total_timesteps=100000)

In [10]:
# 学習した方策で1エピソード実行
evaluate_trained_policy(ppo, task_name)

'openaigym.video.2.72.video000000.mp4'

## A2C｜Advantage Actor-Critic

In [11]:
# CPU コア単位で並列化
pararell_env = make_vec_env(task_name, n_envs=4)

# A2C モデルを作成
a2c = A2C("MlpPolicy", # 方策の種類 
                pararell_env, # 並列化した環境
                learning_rate=0.0007, # 学習率（デフォルト値）
                seed=1234)  # シード値

# 学習（約2分）
a2c.learn(total_timesteps=100000)

In [12]:
# 学習した方策で1エピソード実行
evaluate_trained_policy(a2c, task_name)

'openaigym.video.3.72.video000000.mp4'

## 自作のモデルを使用
- Stable-Baseline 3 では方策関数と価値関数で共有するネットワーク構造を変更することができる
    - 状態を途中まで同じネットワークで処理して，最後の出力層だけ出し分けるという実装になっている
- 2層の全結合NNを使って，A2Cで学習してみよう

In [13]:
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor # 共有ネットワークの親クラス
import torch.nn as nn # PyTorch の層クラス

# 共有ネットワーククラス
class CustomNN(BaseFeaturesExtractor): # 共有ネットワークはBaseFeaturesExtractorを継承する必要がある
    # コンストラクタ
    def __init__(self,
                 observation_space, # 状態の情報が入った変数
                 num_hidden_unit): # 隠れ層のユニット数
        # 親クラスの __init__ 関数を呼び出し（おまじない）
        super(CustomNN, self).__init__(observation_space, num_hidden_unit)
        n_state_dims = observation_space.shape[0] #状態の次元数を取得
        # ネットワークの定義
        self.network = nn.Sequential(
            nn.Linear(n_state_dims, num_hidden_unit), # 全結合層
            nn.Tanh(), # 活性化関数
            nn.Linear(num_hidden_unit, num_hidden_unit), # 全結合層
            nn.Tanh(), # 活性化関数
        )

    # 順伝播
    def forward(self,
                observations): # 状態
        # 共有ネットワークの順伝播の実施
        ## = 方策関数と状態価値にわたす値を生成
        return self.network(observations)

In [14]:
# CPU コア単位で並列化
pararell_env = make_vec_env(task_name, n_envs=4)

# 共有ネットワークの情報をまとめた辞書オブジェクト
policy_kwargs = {
    'features_extractor_class': CustomNN, # 共有ネットワークのクラス名
    'features_extractor_kwargs': {'num_hidden_unit': 128}, # 共有ネットワークの__init__に渡す第二引数以降の値
}

# A2C モデルを作成
a2c_mynn = A2C("MlpPolicy", # 方策の種類 
                pararell_env,  # 並列化した環境
                learning_rate=0.0007, # 学習率（デフォルト値）
                policy_kwargs=policy_kwargs, # 共有ネットワークの情報
                seed=1234)  # シード値

# 学習（約3分）
a2c_mynn.learn(total_timesteps=100000)

In [15]:
# 学習した方策で1エピソード実行
evaluate_trained_policy(a2c_mynn, task_name)

'openaigym.video.4.72.video000000.mp4'